In [1]:
import pandas as pd
import random
import math

Pobranie danych

In [2]:
lirabary = pd.read_csv('books.csv', on_bad_lines='skip')


lirabary.head()


,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals
0,0,Harry Potter and the Half-Blood Prince (Harry ...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,"['travel', 'magical_realism']",single_mothers,"['mafia', 'egyptian_mythology', 'angels', 'sup...",652,eng,France,2006,4.57,2095690,161206
1,1,Harry Potter and the Order of the Phoenix (Har...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,"['parenting_and_families', 'short_story']",young_adults,"['gore', 'egyptian_mythology', 'love_triangle'...",870,eng,Poland,2004,4.49,2153167,165628
2,2,Harry Potter and the Chamber of Secrets (Harry...,['J.K. Rowling'],Scholastic,"['parenting_and_families', 'contemporary_ficti...",boys,"['time_travel', 'gambling']",352,eng,Mexico,2003,4.42,6333,487
3,3,Harry Potter and the Prisoner of Azkaban (Harr...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic Inc.,['young_adult'],introverts,"['time_travel', 'violence', 'slice_of_life']",435,eng,Korea,2004,4.56,2339585,179968
4,4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,"['J.K. Rowling', 'Mary GrandPré']",Scholastic,"['horror', 'action_adventure']",engaged_people,"['egyptian_mythology', 'isekai', 'gods', 'egyp...",2690,eng,Russia,2004,4.78,41428,3186


In [3]:
def Make_List_From_txt(title: str):
    genres = open('additional_data_base_info/' + title + '.txt','r')
    genres_list =[]
    for line in genres:
        genres_list.append(line.strip())
    return genres_list

def ToHamming(all_possibilities, vector):
    result = []
    for x in all_possibilities:
        if(x in vector):
            result.append(1)
        else:
            result.append(0)
    return result

def Haming_Similaryty(vector1:list, vector2:list):
    wynik = 0
    if(len(vector1) != len(vector2)):
        return -1
    
    for i in range(0, len(vector1)-1):
        if(vector1[i] == vector2[i]):
            wynik = wynik +1
    return wynik


Odległość haminga

In [4]:
import numpy as np

genres = Make_List_From_txt("genres")
tags = Make_List_From_txt("tags")

column_to_haming = {"Genres": genres, "tags": tags}

for key in column_to_haming:
    lirabary["haminged_" + key] =  lirabary[key].apply(lambda x: ToHamming(column_to_haming[key], x) )
    


Odległość cosinusowa

In [5]:
import math
from ast import literal_eval
def CosSimilarity(vector_1, book_vector, max_num_pages, max_publicate_data):
    columns_to_cos_metrice = ["publisher", "target_groups", "num_pages", "language_code", "country_of_origin", "publication_date"]
    maxs = {"num_pages": max_num_pages, "publication_date": max_publicate_data}
    squer_sum_bookVector = 0
    squer_sum_vector = 0
    vector_1__multiplay__book_vector = 0

    squer_sum_bookVector += 1
    if any( item in literal_eval(vector_1["authors"]) for item in literal_eval(book_vector["authors"]) ):
        vector_1__multiplay__book_vector += 1
        squer_sum_vector += 1
    
    for name in columns_to_cos_metrice:
        if(type(vector_1[name]) == str):
            squer_sum_bookVector += 1
            if(vector_1[name] == book_vector[name]):
                vector_1__multiplay__book_vector += 1
                squer_sum_vector += 1
        else:
            squer_sum_bookVector += math.pow(book_vector[name] / maxs[name],2)
            squer_sum_vector +=  math.pow(vector_1[name] / maxs[name],2)
            vector_1__multiplay__book_vector += (book_vector[name] * vector_1[name])/(maxs[name]*maxs[name])
    return vector_1__multiplay__book_vector/(math.sqrt(squer_sum_vector*squer_sum_bookVector))








Wyznaczenie biblioteki użytkownika

In [6]:
already_read = lirabary.sample(3)

Wyznaczenie podobieństwa księżek

In [7]:
max_date = lirabary["publication_date"].max()
max_pages = lirabary["num_pages"].max()
genres_lenght = len(lirabary["haminged_Genres"].iloc[0])
tags_lenght = len(lirabary["haminged_tags"].iloc[0])
for index, row in already_read.iterrows():
    lirabary[row["bookID"]] =  lirabary.apply(lambda x:
        CosSimilarity(x, row, max_pages,max_date) + 
        Haming_Similaryty(x["haminged_Genres"], row["haminged_Genres"])/genres_lenght +
        Haming_Similaryty(x["haminged_tags"], row["haminged_tags"])/tags_lenght , axis=1)




Suma metryk

In [8]:
bookid = already_read["bookID"]
lirabary["absolute_metric"] = lirabary[bookid].sum(axis=1)

In [9]:
already_read.iloc[0]

bookID                                                            8507
title                                                    Silver Bullet
authors                           ['Stephen King', 'Bernie Wrightson']
publisher                                                       Signet
Genres               ['supernatural_fiction', 'graphic_novel', 'you...
target_groups                                           engaged_people
tags                                               ['wolves', 'debut']
num_pages                                                          255
language_code                                                    en-US
country_of_origin                                               Russia
publication_date                                                  1985
average_rating                                                    3.64
ratings_count                                                     3551
month_rentals                                                      273
haming

In [10]:
lirabary = lirabary.sort_values(already_read.iloc[0]["bookID"], ascending=False)
lirabary[:5]

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals,haminged_Genres,haminged_tags,8507,385,1723,absolute_metric
8507,8507,Silver Bullet,"['Stephen King', 'Bernie Wrightson']",Signet,"['supernatural_fiction', 'graphic_novel', 'you...",engaged_people,"['wolves', 'debut']",255,en-US,Russia,1985,3.64,3551,273,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.950751,2.083791,2.063323,7.097865
3420,3420,You Don't Know Jack (NY Girlfriends #2),['Erin McCarthy'],Brava,['graphic_novel'],engaged_people,"['gambling', 'archery']",255,en-US,Russia,2006,3.72,2757,212,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.623172,2.137843,2.117370,6.878384
1280,1280,An Unfinished Life: John F. Kennedy 1917-1963,['Robert Dallek'],Back Bay Books,['young_adult'],engaged_people,"['dance', 'betrayal', 'time_travel']",849,en-US,Russia,2004,4.07,7981,613,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",2.600121,2.203327,2.095048,6.898497
5308,5308,Dante Alighieri's Divine Comedy the Inferno,['Dante Alighieri'],Barron's Educational Series,"['history', 'young_adult']",engaged_people,"['betrayal', 'archery']",101,en-US,Russia,1984,4.00,5,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",2.596097,2.164635,2.089832,6.850564
1654,1654,BSD Hacks,['Lavigne'],O'Reilly Media,['economy_and_business'],engaged_people,"['gore', 'aliens']",450,en-US,Russia,2004,3.80,42,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",2.596057,2.164846,2.144729,6.905632


Testowanie finalnego algorytmu

In [11]:
lirabary2 = pd.read_csv('books.csv', on_bad_lines='skip')

sample = lirabary2.sample(10)

import Recomender as ra

result = ra.Recommend(sample)

c:\Users\user\Pulpit\Studia\WSI\books_rekomender\Recomender.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lirabary[row["bookID"]] =  lirabary.apply(lambda x:
c:\Users\user\Pulpit\Studia\WSI\books_rekomender\Recomender.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lirabary["Sum_metrics"] = lirabary[bookid].sum(axis=1)


Porównanie rekomendaci dla pierwszej wylosowanej książki

In [12]:
sample.iloc[0]

bookID                                                           10025
title                                     Swords of Mars (Barsoom  #8)
authors                                       ['Edgar Rice Burroughs']
publisher                                                      Del Rey
Genres                                                   ['self_help']
target_groups                                                  parents
tags                 ['slice_of_life', 'female_protagonist', 'ancie...
num_pages                                                          191
language_code                                                      eng
country_of_origin                                              Ireland
publication_date                                                  1979
average_rating                                                    3.82
ratings_count                                                     2812
month_rentals                                                      216
Name: 

In [13]:
result.sort_values(sample.iloc[0]["bookID"], ascending=False)

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,...,7877,3833,7280,6992,5686,6151,6127,4092,7947,Sum_metrics
10420,10420,Theater Shoes,"['Noel Streatfeild', 'Diane Goode']",Sagebrush,['self_help'],young_adults,"['ancient_egypt', 'ancient_egypt', 'female_pro...",252,en-US,Poland,...,2.743153,2.595351,2.733532,2.617500,2.517877,2.639752,2.419422,2.693864,2.451271,26.270106
2405,2405,Today I Feel Silly Other Moods That Make My Day,"['Jamie Lee Curtis', 'Laura Cornell']",HarperCollins,['self_help'],men,['female_protagonist'],40,en-US,Italy,...,2.765919,2.618190,2.756309,2.640363,2.417463,2.662544,2.443364,2.716711,2.475228,26.332875
9350,9350,鋼の錬金術師 8 (Fullmetal Alchemist 8),['Hiromu Arakawa'],ガンガンコミックス,['science_fiction'],girls,"['time_travel', 'magic', 'slice_of_life', 'fem...",192,jpn,Mexico,...,2.689362,2.551190,2.644909,2.528902,2.694236,2.595578,2.551175,2.649706,2.627386,26.337060
4016,4016,The Atom Station,"['Halldór Laxness', 'Magnus Magnusson']",Vintage Classics,['true_crime'],boys,"['wolves', 'female_protagonist']",192,en-GB,Mexico,...,2.689362,2.595635,2.733798,2.671845,2.738680,2.640022,2.640063,2.694150,2.716275,26.924447
3146,3146,Stinger,['Robert R. McCammon'],Pocket Books,['action_adventure'],mothers,"['male_protagonist', 'wolves', 'ancient_egypt'...",538,en-US,Russia,...,2.686693,2.372392,2.686667,2.614964,2.515492,2.592866,2.328117,2.646903,2.448833,25.694744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8874,8874,Code Word Kangaroo (Adam Sharp #6),"['George E. Stanley', 'Guy Francis']",Random House Books for Young Readers,"['humanities_and_social_sciences', 'parenting_...",parents,"['sex', 'debut', 'gambling', 'time_travel', 'd...",48,eng,Ireland,...,2.354443,2.295563,2.344833,2.146119,2.568969,2.251188,2.524428,2.349600,2.635490,23.675278
7223,7223,Madam Will You Talk?,['Mary Stewart'],HarperTorch,"['religion_and_spirituality', 'magical_realism...",parents,"['educational', 'first_love', 'nordic_mytholog...",360,eng,France,...,2.300008,2.403214,2.261804,2.228447,2.567808,2.358888,2.513610,2.349135,2.634247,23.818066
5406,5406,Moonraker's Bride,['Madeleine Brent'],Souvenir Press,"['mystery', 'historical_fiction', 'guide_how_to']",mothers,"['collective_hero', 'demi-gods', 'ancient_gree...",352,eng,Ireland,...,2.300093,2.384082,2.388971,2.234448,2.572631,2.295312,2.562878,2.438110,2.594627,23.971774
3236,3236,Teach Them Diligently: How to Use the Scriptur...,['Lou Priolo'],Timeless Texts,"['for_children', 'magical_realism', 'historica...",parents,"['superhero', 'egyptian_mythology', 'superhero...",162,eng,Ireland,...,2.322571,2.228829,2.367011,2.257192,2.600635,2.228917,2.447969,2.371752,2.568628,23.575907


In [6]:
import Recomender as ra
ra.TheBestBooks(0)

,bookID,title,authors,publisher,Genres,target_groups,tags,num_pages,language_code,country_of_origin,publication_date,average_rating,ratings_count,month_rentals
624,624,Comoediae 1: Acharenses/Equites/Nubes/Vespae/P...,"['Aristophanes', 'F.W. Hall', 'W.M. Geldart']",Oxford University Press USA,['graphic_novel'],boys,"['nordic_mythology', 'gambling']",364,grc,Brazil,1922,5.0,0,0
9893,9893,His Princess Devotional: A Royal Encounter Wit...,['Sheri Rose Shepherd'],Multnomah,"['memoir_and_autobiography', 'graphic_novel', ...",pregnant_people,"['dance', 'wolves', 'violence', 'the_chosen_on...",240,eng,Japan,2007,5.0,2,0
4788,4788,The Diamond Color Meditation: Color Pathway to...,['John Diamond'],Square One Publishers,"['science_fiction', 'science_fiction']",married_people,['demons'],74,eng,Germany,2006,5.0,5,0
9324,9324,Tyrannosaurus Wrecks (Stanley #1),"['Laura Driscoll', 'Alisa Klayman-Grodsky', 'E...",Disney Press,"['diets', 'contemporary_fiction']",girls,"['archery', 'archery', 'chinese_mythology', 'l...",24,eng,Mexico,2003,5.0,2,0
9720,9720,The Irish Anatomist: A Study of Flann O'Brien,['Keith Donohue'],Academica Press,"['graphic_novel', 'women_s_fiction']",pregnant_people,"['slice_of_life', 'female_protagonist', 'fatal...",222,eng,Japan,2003,5.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,6877,I'll Be Home Before Midnight and I Won't Get P...,['Anthony E. Wolf'],Knopf Doubleday Publishing Group,['food_and_drinks'],young_adults,"['time_travel', 'wolves', 'egyptian_mythology'...",288,eng,Poland,1988,0.0,0,0
3492,3492,Brodie's notes on Aldous Huxley's brave new world,['Graham Handley'],Macmillan,['fantasy'],adults,"['debut', 'vampires', 'nordic_mythology', 'vik...",71,eng,United_Kingdom,1992,0.0,0,0
2533,2533,Insights: Talks On The Nature Of Existence,['Frederick P. Lenz'],Frederick P. Lenz Foundation for American Budd...,['essays'],married_people,"['war', 'egyptian_mythology', 'ghosts', 'slice...",304,eng,Germany,2003,0.0,0,0
2532,2532,How To Meditate: An Anthology Of Talks On Medi...,['Frederick P. Lenz'],Frederick P. Lenz Foundation for American Budd...,"['parenting_and_families', 'true_crime', 'hist...",married_people,"['chinese_mythology', 'ghosts']",228,eng,Germany,2004,0.0,0,0
